In [ ]:
#Metadata

In [2]:
from qiime2 import Metadata
from urllib import request

url = 'https://docs.qiime2.org/jupyterbooks/cancer-microbiome-intervention-tutorial/data/020-tutorial-upstream/020-metadata/sample-metadata.tsv'
fn = 'sample-metadata.tsv'
request.urlretrieve(url, fn)
sample_metadata_md = Metadata.load(fn)

In [ ]:
import qiime2.plugins.metadata.actions as metadata_actions

metadata_summ_1_viz, = metadata_actions.tabulate(input=sample_metadata_md,)

In [6]:
#Demultiplex

In [ ]:
import zipfile

url = 'https://docs.qiime2.org/jupyterbooks/cancer-microbiome-intervention-tutorial/data/020-tutorial-upstream/030-importing/data_to_import.zip'
fn = 'data_to_import.zip'
request.urlretrieve(url, fn)
with zipfile.ZipFile(fn) as zf:
    zf.extractall('data_to_import')

In [ ]:
from q2_types.per_sample_sequences import CasavaOneEightSingleLanePerSampleDirFmt
from qiime2 import Artifact

demultiplexed_sequences = Artifact.import_data(
    'SampleData[PairedEndSequencesWithQuality]',
    'data_to_import',
    CasavaOneEightSingleLanePerSampleDirFmt,)

In [ ]:
import qiime2.plugins.demux.actions as demux_actions

demultiplexed_sequences_summ_viz, = demux_actions.summarize(
    data=demultiplexed_sequences,)

In [ ]:
#Denoise

In [ ]:
import qiime2.plugins.dada2.actions as dada2_actions

feature_table_0, asv_sequences_0, dada2_stats = dada2_actions.denoise_paired(
    demultiplexed_seqs=demultiplexed_sequences,
    trunc_len_f=204,
    trim_left_r=1,
    trunc_len_r=205,)

In [ ]:
stats_dada2_md_md = dada2_stats.view(Metadata)
dada2_stats_summ_viz, = metadata_actions.tabulate(
    input=stats_dada2_md_md,)

In [8]:
import qiime2.plugins.feature_table.actions as feature_table_actions

feature_table_0_summ_viz, = feature_table_actions.summarize(
    table=feature_table_0,
    sample_metadata=sample_metadata_md,)

asv_sequences_0_summ_viz, = feature_table_actions.tabulate_seqs(
    data=asv_sequences_0,)

In [ ]:
#Filter
#jupyter serverextension enable --py qiime2 --sys-prefix needed fro visualization

In [14]:
url = 'https://docs.qiime2.org/jupyterbooks/cancer-microbiome-intervention-tutorial/data/030-tutorial-downstream/010-filtering/feature-table.qza'
fn = 'feature-table.qza'
request.urlretrieve(url, fn)
feature_table = Artifact.load(fn)

In [15]:
url = 'https://docs.qiime2.org/jupyterbooks/cancer-microbiome-intervention-tutorial/data/030-tutorial-downstream/010-filtering/rep-seqs.qza'
fn = 'rep-seqs.qza'
request.urlretrieve(url, fn)
rep_seqs = Artifact.load(fn)

In [27]:
table_viz, = feature_table_actions.summarize(
    table=feature_table,
    sample_metadata=sample_metadata_md,)
table_viz.save('table.qzv')

rep_seqs_viz, = feature_table_actions.tabulate_seqs(
    data=rep_seqs,)
rep_seqs_viz.save('resp_seqs.qzv')

In [11]:
table_viz

<visualization: Visualization uuid: b0c159a8-df76-4fd4-9f8d-46d96c5abe8c>

In [15]:
rep_seqs_viz

<visualization: Visualization uuid: 509f7752-0f06-438b-b997-7f15968c06e7>

In [16]:
autofmt_table, = feature_table_actions.filter_samples(
    table=feature_table,
    metadata=sample_metadata_md,
    where='autoFmtGroup IS NOT NULL',)

In [17]:
autofmt_table.save('autofmt-table.qza')

'autofmt-table.qza'

In [18]:
autofmt_table_summ_viz, = feature_table_actions.summarize(
    table=autofmt_table,
    sample_metadata=sample_metadata_md,)

In [19]:
autofmt_table_summ_viz.save('autofmt-table-summ.qzv')

'autofmt-table-summ.qzv'

In [20]:
autofmt_table_summ_viz

<visualization: Visualization uuid: 13642732-6850-400d-8669-c6cc88641c90>

In [21]:
filtered_table_1, = feature_table_actions.filter_samples(
    table=autofmt_table,
    metadata=sample_metadata_md,
    where='DayRelativeToNearestHCT BETWEEN -10 AND 70',)

In [22]:
filtered_table_1.save('filtered_table_1.qza')

'filtered_table_1.qza'

In [23]:
filtered_table_2, = feature_table_actions.filter_features(
    table=filtered_table_1,
    min_samples=2,)

In [24]:
filtered_table_2.save('filtered_table_2.qza')

'filtered_table_2.qza'

In [25]:
filtered_sequences_1, = feature_table_actions.filter_seqs(
    data=rep_seqs,
    table=filtered_table_2,)

/opt/miniconda3/envs/qiime2a/lib/python3.8/site-packages/q2_types/feature_data/_transformer.py:258: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for id_, seq in data.iteritems():


In [26]:
filtered_sequences_1.save('filtered_sequences_1.qza')

'filtered_sequences_1.qza'

In [ ]:
#Taxonomy

In [27]:
url = 'https://docs.qiime2.org/jupyterbooks/cancer-microbiome-intervention-tutorial/data/030-tutorial-downstream/020-taxonomy/gg-13-8-99-nb-classifier.qza'
fn = 'gg-13-8-99-nb-classifier.qza'
request.urlretrieve(url, fn)
gg_13_8_99_nb_classifier = Artifact.load(fn)

In [28]:
import qiime2.plugins.feature_classifier.actions as feature_classifier_actions

taxonomy, = feature_classifier_actions.classify_sklearn(
    classifier=gg_13_8_99_nb_classifier,
    reads=filtered_sequences_1,)

In [29]:
taxonomy.save('taxonomy.qza')

'taxonomy.qza'

In [32]:
taxonomy_as_md_md = taxonomy.view(Metadata)
taxonomy_viz, = metadata_actions.tabulate(
    input=taxonomy_as_md_md,)

In [33]:
taxonomy_viz.save('taxonomy.qzv')

'taxonomy.qzv'

In [35]:
taxonomy_viz

<visualization: Visualization uuid: 94ec8592-7f97-4807-bb30-6434f3d502ab>

In [36]:
import qiime2.plugins.taxa.actions as taxa_actions

filtered_table_3, = taxa_actions.filter_table(
    table=filtered_table_2,
    taxonomy=taxonomy,
    mode='contains',
    include='p__',
    exclude='p__;,Chloroplast,Mitochondria',)

In [37]:
filtered_table_3.save('filtered_table_3.qza')

'filtered_table_3.qza'

In [38]:
filtered_table_4, = feature_table_actions.filter_samples(
    table=filtered_table_3,
    min_frequency=10000,)

In [39]:
filtered_table_4.save('filtered_table_4.qza')

'filtered_table_4.qza'

In [41]:
filtered_sequences_2, = feature_table_actions.filter_seqs(
    data=rep_seqs,
    table=filtered_table_4,)

/opt/miniconda3/envs/qiime2a/lib/python3.8/site-packages/q2_types/feature_data/_transformer.py:258: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for id_, seq in data.iteritems():


In [42]:
filtered_sequences_2.save('filtered_sequences_2.qza')

'filtered_sequences_2.qza'

In [43]:
taxa_bar_plots_1_viz, = taxa_actions.barplot(
    table=filtered_table_4,
    taxonomy=taxonomy,
    metadata=sample_metadata_md,)

In [44]:
taxa_bar_plots_1_viz.save('taxa_bar_plots_1.qzv')

'taxa_bar_plots_1.qzv'

In [45]:
taxa_bar_plots_1_viz

<visualization: Visualization uuid: 228e9833-9db3-4fd2-92f0-8f8b974da723>

In [ ]:
#Phylogeny

In [46]:
import qiime2.plugins.phylogeny.actions as phylogeny_actions

action_results = phylogeny_actions.align_to_tree_mafft_fasttree(
    sequences=filtered_sequences_2,)

aligned_rep_seqs = action_results.alignment
masked_aligned_rep_seqs = action_results.masked_alignment
unrooted_tree = action_results.tree
rooted_tree = action_results.rooted_tree

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: mafft --preservecase --inputorder --thread 1 /var/folders/hq/mt_9wjp90cx45qsgnk3d7qfc0000gq/T/qiime2/reziw3/data/4aa1fd46-786d-401d-b7d5-34c8792b3655/data/dna-sequences.fasta



inputfile = orig
2429 x 340 - 321 d
nthread = 1
nthreadpair = 1
nthreadtb = 1
ppenalty_ex = 0
stacksize: 8192 kb
generating a scoring matrix for nucleotide (dist=200) ... done
Gap Penalty = -1.53, +0.00, +0.00



Making a distance matrix ..
 2401 / 2429 (thread    0)
done.

Constructing a UPGMA tree (efffree=0) ... 
 2420 / 2429
done.

Progressive alignment 1/2... 
STEP  2301 / 2428 (thread    0)
Reallocating..done. *alloclen = 1684
STEP  2401 / 2428 (thread    0)
done.

Making a distance matrix from msa.. 
 2400 / 2429 (thread    0)
done.

Constructing a UPGMA tree (efffree=1) ... 
 2420 / 2429
done.

Progressive alignment 2/2... 
STEP  2301 / 2428 (thread    0)
Reallocating..done. *alloclen = 1684
STEP  2401 / 2428 (thread    0)
done.

disttbfast (nuc) Version 7.520
alg=A, model=DNA200 (2), 1.53 (4.59), -0.00 (-0.00), noshift, amax=0.0
1 thread(s)


Strategy:
 FFT-NS-2 (Fast but rough)
 Progressive method (guide trees were built 2 times.)

If unsure which option to use, try 'mafft --

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: FastTree -quote -nt /var/folders/hq/mt_9wjp90cx45qsgnk3d7qfc0000gq/T/qiime2/reziw3/data/d829c65f-16c4-4e7c-afb6-b4dfb2b94e9c/data/aligned-dna-sequences.fasta



FastTree Version 2.1.11 Double precision (No SSE3)
Alignment: /var/folders/hq/mt_9wjp90cx45qsgnk3d7qfc0000gq/T/qiime2/reziw3/data/d829c65f-16c4-4e7c-afb6-b4dfb2b94e9c/data/aligned-dna-sequences.fasta
Nucleotide distances: Jukes-Cantor Joins: balanced Support: SH-like 1000
Search: Normal +NNI +SPR (2 rounds range 10) +ML-NNI opt-each=1
TopHits: 1.00*sqrtN close=default refresh=0.80
ML Model: Jukes-Cantor, CAT approximation with 20 rate categories
      0.10 seconds: Top hits for    916 of   2389 seqs (at seed    400)
      0.20 seconds: Top hits for   1694 of   2389 seqs (at seed   1200)
      0.30 seconds: Top hits for   2379 of   2389 seqs (at seed   2300)
      0.47 seconds: Joined    200 of   2386
      0.60 seconds: Joined    300 of   2386
      0.77 seconds: Joined    500 of   2386
      0.93 seconds: Joined    700 of   2386
      1.07 seconds: Joined    900 of   2386
      1.24 seconds: Joined   1100 of   2386
      1.41 seconds: Joined   1300 of   2386
      1.52 seconds: Joined

     11.54 seconds: ML NNI round 9 of 22, 1901 of 2387 splits, 22 changes (max delta 3.625)
     11.68 seconds: ML NNI round 9 of 22, 2101 of 2387 splits, 24 changes (max delta 4.285)
     11.80 seconds: ML NNI round 9 of 22, 2301 of 2387 splits, 25 changes (max delta 4.285)
ML-NNI round 9: LogLk = -76471.593 NNIs 25 max delta 4.28 Time 11.86 (final)
     11.91 seconds: ML Lengths 401 of 2387 splits
     12.03 seconds: ML Lengths 1201 of 2387 splits
     12.13 seconds: ML Lengths 1901 of 2387 splits
Optimize all lengths: LogLk = -76465.607 Time 12.22
     12.32 seconds: ML split tests for    100 of   2386 internal splits
     12.51 seconds: ML split tests for    300 of   2386 internal splits
     12.62 seconds: ML split tests for    400 of   2386 internal splits
     12.81 seconds: ML split tests for    600 of   2386 internal splits
     12.91 seconds: ML split tests for    700 of   2386 internal splits
     13.10 seconds: ML split tests for    900 of   2386 internal splits
     13.29 

In [49]:
aligned_rep_seqs.save('phylogeny/alignment.qza')
masked_aligned_rep_seqs.save('phylogeny/masked_alignment.qza')
unrooted_tree.save('phylogeny/tree.qza')
rooted_tree.save('phylogeny/rooted_tree.qza')

'phylogeny/rooted_tree.qza'

In [ ]:
#Sampling depth

In [51]:
filtered_table_4_summ_viz, = feature_table_actions.summarize(
    table=filtered_table_4,
    sample_metadata=sample_metadata_md,)

In [52]:
filtered_table_4_summ_viz.save('filtered_table_4_summ.qzv')

'filtered_table_4_summ.qzv'

In [53]:
import qiime2.plugins.diversity.actions as diversity_actions

shannon_rarefaction_plot_viz, = diversity_actions.alpha_rarefaction(
    table=filtered_table_4,
    metrics={'shannon'},
    metadata=sample_metadata_md,
    max_depth=33000,)

/opt/miniconda3/envs/qiime2a/lib/python3.8/site-packages/q2_diversity/_alpha/_visualizer.py:245: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  median_ = grouped.median()
/opt/miniconda3/envs/qiime2a/lib/python3.8/site-packages/q2_diversity/_alpha/_visualizer.py:245: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  median_ = grouped.median()
/opt/miniconda3/envs/qiime2a/lib/python3.8/site-packages/q2_diversity/_alpha/_visualizer.py:245: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_on

In [58]:
shannon_rarefaction_plot_viz.save('shannon_rarefaction_plot.qzv')

'shannon_rarefaction_plot.qzv'

In [62]:
shannon_rarefaction_plot_viz

<visualization: Visualization uuid: bf8f0bad-d65e-40f4-a5f9-50da5448f84e>

In [56]:
braycurtis_rarefaction_plot_viz, = diversity_actions.beta_rarefaction(
    table=filtered_table_4,
    metric='braycurtis',
    clustering_method='nj',
    sampling_depth=10000,
    metadata=sample_metadata_md,)

/opt/miniconda3/envs/qiime2a/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.6330448261356202 and the largest is 22.365742026405208.
  warn(
/opt/miniconda3/envs/qiime2a/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest e

In [57]:
braycurtis_rarefaction_plot_viz.save('braycurtis_rarefaction_plot.qzv')

'braycurtis_rarefaction_plot.qzv'

In [60]:
braycurtis_rarefaction_plot_viz

<visualization: Visualization uuid: dbf50e36-ccd5-4ea1-abf5-615d4b07cb6f>

In [ ]:
#Computing diversity metrics

In [20]:
action_results = diversity_actions.core_metrics_phylogenetic(
    phylogeny=rooted_tree,
    table=filtered_table_4,
    sampling_depth=10000,
    metadata=sample_metadata_md,)

rarefied_table = action_results.rarefied_table
faith_pd_vector = action_results.faith_pd_vector
observed_features_vector = action_results.observed_features_vector
shannon_vector = action_results.shannon_vector
evenness_vector = action_results.evenness_vector
unweighted_unifrac_distance_matrix = action_results.unweighted_unifrac_distance_matrix
weighted_unifrac_distance_matrix = action_results.weighted_unifrac_distance_matrix
jaccard_distance_matrix = action_results.jaccard_distance_matrix
bray_curtis_distance_matrix = action_results.bray_curtis_distance_matrix
unweighted_unifrac_pcoa_results = action_results.unweighted_unifrac_pcoa_results
weighted_unifrac_pcoa_results = action_results.weighted_unifrac_pcoa_results
jaccard_pcoa_results = action_results.jaccard_pcoa_results
bray_curtis_pcoa_results = action_results.bray_curtis_pcoa_results
unweighted_unifrac_emperor_viz = action_results.unweighted_unifrac_emperor
weighted_unifrac_emperor_viz = action_results.weighted_unifrac_emperor
jaccard_emperor_viz = action_results.jaccard_emperor
bray_curtis_emperor_viz = action_results.bray_curtis_emperor

/opt/miniconda3/envs/qiime2a/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.021204273898313482 and the largest is 9.860404378772003.
  warn(
/opt/miniconda3/envs/qiime2a/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest 

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command:

faithpd -i /var/folders/hq/mt_9wjp90cx45qsgnk3d7qfc0000gq/T/qiime2/reziw3/data/e6cb807f-a7ce-4b20-bc85-75cf129f616b/data/feature-table.biom -t /var/folders/hq/mt_9wjp90cx45qsgnk3d7qfc0000gq/T/qiime2/reziw3/data/3f75b1f3-c11b-4f26-86e6-f22e7d970530/data/tree.nwk -o /var/folders/hq/mt_9wjp90cx45qsgnk3d7qfc0000gq/T/q2-AlphaDiversityFormat-lcvtw0sp

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command:

ssu -i /var/folders/hq/mt_9wjp90cx45qsgnk3d7qfc0000gq/T/qiime2/reziw3/data/e6cb807f-a7ce-4b20-bc85-75cf129f616b/data/feature-table.biom -t /var/folders/hq/mt_9wjp90cx45qsgnk3d7qfc0000gq/

/opt/miniconda3/envs/qiime2a/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.06880147610415145 and the largest is 13.698096614841239.
  warn(
/opt/miniconda3/envs/qiime2a/lib/python3.8/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:143: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest 

In [66]:
rarefied_table.save('diversity/rarefied_table.qza')
faith_pd_vector.save('diversity/faith_pd_vector.qza')
observed_features_vector.save('diversity/observed_features_vector.qza')
shannon_vector.save('diversity/shannon_vector.qza')
evenness_vector.save('diversity/evenness_vector.qza')
unweighted_unifrac_distance_matrix.save('diversity/unweighted_unifrac_distance_matrix.qza')
weighted_unifrac_distance_matrix.save('diversity/weighted_unifrac_distance_matrix.qza')
jaccard_distance_matrix.save('diversity/jaccard_distance_matrix.qza')
bray_curtis_distance_matrix.save('diversity/bray_curtis_distance_matrix.qza')
unweighted_unifrac_pcoa_results.save('diversity/unweighted_unifrac_pcoa_results.qza')
weighted_unifrac_pcoa_results.save('diversity/weighted_unifrac_pcoa_results.qza')
jaccard_pcoa_results.save('diversity/jaccard_pcoa_results.qza')
bray_curtis_pcoa_results.save('diversity/bray_curtis_pcoa_results.qza')
unweighted_unifrac_emperor_viz.save('diversity/unweighted_unifrac_emperor.qzv')
weighted_unifrac_emperor_viz.save('diversity/weighted_unifrac_emperor.qzv')
jaccard_emperor_viz.save('diversity/jaccard_emperor.qzv')
bray_curtis_emperor_viz.save('diversity/bray_curtis_emperor.qzv')

'diversity/bray_curtis_emperor.qzv'

In [70]:
bray_curtis_emperor_viz

<visualization: Visualization uuid: 411ff9a0-2901-4f23-a453-094761a81853>

In [29]:
from qiime2 import Artifact
faith_pd_vector=Artifact.load('diversity/faith_pd_vector.qza')
evenness_vector=Artifact.load('diversity/evenness_vector.qza')

In [30]:
faith_pd_group_significance_viz, = diversity_actions.alpha_group_significance(
    alpha_diversity=faith_pd_vector,
    metadata=sample_metadata_md,)

evenness_group_significance_viz, = diversity_actions.alpha_group_significance(
    alpha_diversity=evenness_vector,
    metadata=sample_metadata_md,)

In [31]:
faith_pd_group_significance_viz.save('faith_pd_group_significance.qzv')
evenness_group_significance_viz.save('evenness_group_significance.qzv')

'evenness_group_significance.qzv'

In [32]:
evenness_group_significance_viz

<visualization: Visualization uuid: 4d917894-dd1d-4dec-a016-2c3a302a4b0d>

In [28]:
faith_pd_group_significance_viz

<visualization: Visualization uuid: 501ababa-e109-4403-a012-8c6593910148>

In [39]:
hctsource_mdc = sample_metadata_md.get_column('HCTSource')
unweighted_unifrac_disease_group_significance_viz, = diversity_actions.beta_group_significance(
    distance_matrix=unweighted_unifrac_distance_matrix,
    metadata=hctsource_mdc,
    pairwise=True,)

In [48]:
unweighted_unifrac_disease_group_significance_viz.save('unweighted_unifrac_disease_group_significance.qzv')

'unweighted_unifrac_disease_group_significance.qzv'

In [40]:
unweighted_unifrac_disease_group_significance_viz

<visualization: Visualization uuid: be93496a-ed2d-4a88-b6b0-96ea4953af8a>

In [46]:
from qiime2 import Visualization
filtered_table_4_summ_viz=Visualization.load('filtered_table_4_summ.qzv')

In [47]:
filtered_table_4_summ_viz

<visualization: Visualization uuid: d1eaade5-84dd-46d2-9d8a-7d791c0fb8f5>

In [ ]:
#Alpha diversity

In [71]:
alpha_group_sig_obs_feats_viz, = diversity_actions.alpha_group_significance(
    alpha_diversity=observed_features_vector,
    metadata=sample_metadata_md,)

In [72]:
alpha_group_sig_obs_feats_viz.save('alpha-group-sig-obs-feats.qzv')

'alpha-group-sig-obs-feats.qzv'

In [73]:
alpha_group_sig_obs_feats_viz

<visualization: Visualization uuid: b77fcd97-70bf-4c9e-b001-388ab2157e14>

In [74]:
import qiime2.plugins.longitudinal.actions as longitudinal_actions

md_observed_features_md = observed_features_vector.view(Metadata)
merged_observed_features_md = sample_metadata_md.merge(md_observed_features_md)
lme_obs_features_HCT_viz, = longitudinal_actions.linear_mixed_effects(
    metadata=merged_observed_features_md,
    state_column='DayRelativeToNearestHCT',
    individual_id_column='PatientID',
    metric='observed_features',)

/opt/miniconda3/envs/qiime2a/lib/python3.8/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/opt/miniconda3/envs/qiime2a/lib/python3.8/site-packages/statsmodels/regression/mixed_linear_model.py:2201: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
/opt/miniconda3/envs/qiime2a/lib/python3.8/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/opt/miniconda3/envs/qiime2a/lib/python3.8/site-packages/statsmodels/regression/mixed_linear_model.py:2201: ConvergenceWarning: Retrying MixedLM optimization with cg
  warnings.warn(
/opt/miniconda3/envs/qiime2a/lib/python3.8/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to con

In [75]:
lme_obs_features_HCT_viz.save('lme_obs_features_HCT.qzv')

'lme_obs_features_HCT.qzv'

In [76]:
lme_obs_features_HCT_viz

<visualization: Visualization uuid: 5dd621b6-559e-442b-8ddf-707e272d03ee>

In [80]:
lme_obs_features_FMT_viz, = longitudinal_actions.linear_mixed_effects(
    metadata=merged_observed_features_md,
    state_column='day-relative-to-fmt',
    individual_id_column='PatientID',
    metric='observed_features',)

/opt/miniconda3/envs/qiime2a/lib/python3.8/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/opt/miniconda3/envs/qiime2a/lib/python3.8/site-packages/statsmodels/regression/mixed_linear_model.py:2201: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
/opt/miniconda3/envs/qiime2a/lib/python3.8/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/opt/miniconda3/envs/qiime2a/lib/python3.8/site-packages/statsmodels/regression/mixed_linear_model.py:2201: ConvergenceWarning: Retrying MixedLM optimization with cg
  warnings.warn(
/opt/miniconda3/envs/qiime2a/lib/python3.8/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to con

In [81]:
lme_obs_features_FMT_viz.save('lme_obs_features_FMT.qzv')

'lme_obs_features_FMT.qzv'

In [82]:
lme_obs_features_FMT_viz

<visualization: Visualization uuid: 772ce856-e0bd-4ea3-a5bd-51714ba532af>

In [77]:
lme_obs_features_treatmentVScontrol_viz, = longitudinal_actions.linear_mixed_effects(
    metadata=merged_observed_features_md,
    state_column='day-relative-to-fmt',
    group_columns='autoFmtGroup',
    individual_id_column='PatientID',
    metric='observed_features',)

/opt/miniconda3/envs/qiime2a/lib/python3.8/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/opt/miniconda3/envs/qiime2a/lib/python3.8/site-packages/statsmodels/regression/mixed_linear_model.py:2201: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
/opt/miniconda3/envs/qiime2a/lib/python3.8/site-packages/q2_longitudinal/_longitudinal.py:327: UserWarning: This is only a warning, and the results of this action are still valid. The column name "predicted observed_features" already exists in your metadata file. Any "raw" metadata that can be downloaded from the resulting visualization will contain overwritten values for this metadata column, not the original values.
  warnings.warn(warning, UserWarning)


In [78]:
lme_obs_features_treatmentVScontrol_viz.save('lme_obs_features_treatmentVScontrol.qzv')

'lme_obs_features_treatmentVScontrol.qzv'

In [79]:
lme_obs_features_treatmentVScontrol_viz

<visualization: Visualization uuid: 88c5d17c-a02a-4c50-a094-a30c9c892002>

In [ ]:
#Beta diversity

In [83]:
uu_umap, = diversity_actions.umap(
    distance_matrix=unweighted_unifrac_distance_matrix,)

wu_umap, = diversity_actions.umap(
    distance_matrix=weighted_unifrac_distance_matrix,)

/opt/miniconda3/envs/qiime2a/lib/python3.8/site-packages/q2_diversity/_ordination.py:103: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  proportion_explained=pd.Series(None, index=axis_labels),
/opt/miniconda3/envs/qiime2a/lib/python3.8/site-packages/q2_diversity/_ordination.py:103: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  proportion_explained=pd.Series(None, index=axis_labels),


In [84]:
uu_umap.save('uu_umap.qza')
wu_umap.save('wu_umap.qza')

'wu_umap.qza'

In [85]:
uu_umap_as_metadata_md = uu_umap.view(Metadata)
faith_pd_as_metadata_md = faith_pd_vector.view(Metadata)
evenness_as_metadata_md = evenness_vector.view(Metadata)
shannon_as_metadata_md = shannon_vector.view(Metadata)
expanded_sample_metadata_md = sample_metadata_md.merge(uu_umap_as_metadata_md, faith_pd_as_metadata_md, evenness_as_metadata_md, shannon_as_metadata_md)

In [86]:
expanded_metadata_summ_viz, = metadata_actions.tabulate(
    input=expanded_sample_metadata_md,)

In [87]:
expanded_metadata_summ_viz.save('expanded_metadata_summ.qzv')

'expanded_metadata_summ.qzv'

In [88]:
expanded_metadata_summ_viz

<visualization: Visualization uuid: c90bf3ff-252a-418e-83bc-06970b852346>

In [89]:
taxa_bar_plots_2_viz, = taxa_actions.barplot(
    table=filtered_table_4,
    taxonomy=taxonomy,
    metadata=expanded_sample_metadata_md,)

In [90]:
taxa_bar_plots_2_viz.save('taxa_bar_plots_2.qzv')

'taxa_bar_plots_2.qzv'

In [91]:
taxa_bar_plots_2_viz

<visualization: Visualization uuid: 45000cf7-d7fb-4988-b8a4-c11178391d5a>

In [92]:
import qiime2.plugins.emperor.actions as emperor_actions

uu_umap_emperor_w_time_viz, = emperor_actions.plot(
    pcoa=uu_umap,
    metadata=expanded_sample_metadata_md,
    custom_axes=['week-relative-to-hct'],)

wu_umap_emperor_w_time_viz, = emperor_actions.plot(
    pcoa=wu_umap,
    metadata=expanded_sample_metadata_md,
    custom_axes=['week-relative-to-hct'],)

uu_pcoa_emperor_w_time_viz, = emperor_actions.plot(
    pcoa=unweighted_unifrac_pcoa_results,
    metadata=expanded_sample_metadata_md,
    custom_axes=['week-relative-to-hct'],)

wu_pcoa_emperor_w_time_viz, = emperor_actions.plot(
    pcoa=weighted_unifrac_pcoa_results,
    metadata=expanded_sample_metadata_md,
    custom_axes=['week-relative-to-hct'],)

In [93]:
uu_umap_emperor_w_time_viz.save('uu_umap_emperor_w_time.qzv')
wu_umap_emperor_w_time_viz.save('wu_umap_emperor_w_time.qzv')
uu_pcoa_emperor_w_time_viz.save('uu_pcoa_emperor_w_time.qzv')
wu_pcoa_emperor_w_time_viz.save('wu_pcoa_emperor_w_time.qzv')

'wu_pcoa_emperor_w_time.qzv'

In [95]:
wu_umap_emperor_w_time_viz

<visualization: Visualization uuid: 6cafa853-7331-4118-a1a9-b87955f5820e>

In [97]:
wu_pcoa_emperor_w_time_viz

<visualization: Visualization uuid: a70d2567-0a05-4c56-842b-88bdac96f80e>

In [ ]:
#Longitudinal microbiome analysis

In [99]:
genus_table, = taxa_actions.collapse(
    table=filtered_table_4,
    taxonomy=taxonomy,
    level=6,)

In [100]:
genus_table.save('genus_table.qza')

'genus_table.qza'

In [101]:
filtered_genus_table, = feature_table_actions.filter_features_conditionally(
    table=genus_table,
    prevalence=0.1,
    abundance=0.01,)

In [102]:
filtered_genus_table.save('filtered_genus_table.qza')

'filtered_genus_table.qza'

In [105]:
genus_rf_table, = feature_table_actions.relative_frequency(
    table=filtered_genus_table,)

In [106]:
genus_rf_table.save('genus_rf_table.qza')

'genus_rf_table.qza'

In [107]:
volatility_plot_1_viz, = longitudinal_actions.volatility(
    table=genus_rf_table,
    state_column='week-relative-to-hct',
    metadata=expanded_sample_metadata_md,
    individual_id_column='PatientID',
    default_group_column='autoFmtGroup',)

In [108]:
volatility_plot_1_viz.save('volatility_plot_1.qzv')

'volatility_plot_1.qza.qzv'

In [109]:
volatility_plot_1_viz

<visualization: Visualization uuid: 67e27ee6-1f20-451a-839f-a46076c9f66c>

In [110]:
volatility_plot_2_viz, = longitudinal_actions.volatility(
    table=genus_rf_table,
    state_column='week-relative-to-fmt',
    metadata=expanded_sample_metadata_md,
    individual_id_column='PatientID',
    default_group_column='autoFmtGroup',)

In [111]:
volatility_plot_2_viz.save('volatility_plot_2.qzv')

'volatility_plot_2.qzv'

In [113]:
volatility_plot_2_viz

<visualization: Visualization uuid: 27a3a86b-baea-4c48-ae87-ffc5ff1b577b>

In [114]:
action_results = longitudinal_actions.feature_volatility(
    table=filtered_genus_table,
    metadata=expanded_sample_metadata_md,
    state_column='week-relative-to-hct',
    individual_id_column='PatientID',)

important_genera_table_1 = action_results.filtered_table
genus_importances_1 = action_results.feature_importance
genus_volatility_plot_1_viz = action_results.volatility_plot
accuracy_results_1_viz = action_results.accuracy_results
sample_estimator_1 = action_results.sample_estimator

In [117]:
important_genera_table_1.save('longitudinal/filtered_table.qza')
genus_importances_1.save('longitudinal/feature_importance.qza')
genus_volatility_plot_1_viz.save('longitudinal/volatility_plot.qzv')
accuracy_results_1_viz.save('longitudinal/accuracy_results.qzv')
sample_estimator_1.save('longitudinal/sample_estimator.qza')

'longitudinal/sample_estimator.qza'

In [118]:
genus_volatility_plot_1_viz

<visualization: Visualization uuid: ba7aee0e-ef38-4a29-b076-e8018ac5410c>

In [119]:
accuracy_results_1_viz

<visualization: Visualization uuid: 76c3894b-bd50-4935-bf3c-e66cc56c3534>

In [120]:
action_results = longitudinal_actions.feature_volatility(
    table=filtered_genus_table,
    metadata=expanded_sample_metadata_md,
    state_column='week-relative-to-fmt',
    individual_id_column='PatientID',)

important_genera_table_2 = action_results.filtered_table
genus_importances_2 = action_results.feature_importance
genus_volatility_plot_2_viz = action_results.volatility_plot
accuracy_results_2_viz = action_results.accuracy_results
sample_estimator_2 = action_results.sample_estimator

In [121]:
important_genera_table_2.save('longitudinal2/filtered_table.qza')
genus_importances_2.save('longitudinal2/feature_importance.qza')
genus_volatility_plot_2_viz.save('longitudinal2/volatility_plot.qzv')
accuracy_results_2_viz.save('longitudinal2/accuracy_results.qzv')
sample_estimator_2.save('longitudinal2/sample_estimator.qza')

'longitudinal2/sample_estimator.qza'

In [122]:
genus_volatility_plot_2_viz

<visualization: Visualization uuid: b350165a-5968-4716-bf54-9a269bd026b4>

In [123]:
accuracy_results_2_viz

<visualization: Visualization uuid: 8dc1d3cb-ca56-4f75-913a-434d4ee9b9f9>

In [ ]:
#Differential abundance

In [52]:
import qiime2.plugins.feature_table.actions as feature_table_actions
cord_table, = feature_table_actions.filter_samples(
    table=filtered_table_4,
    metadata=sample_metadata_md,
    where='[HCTSource]="cord"',)

In [53]:
cord_table.save('cord_table.qza')

'cord_table.qza'

In [62]:
import qiime2.plugins.composition.actions as composition_actions

ancombc_subject, = composition_actions.ancombc(
    table=cord_table,
    metadata=sample_metadata_md,
    formula='RandomizationArm',)

da_barplot_subject_viz, = composition_actions.da_barplot(
    data=ancombc_subject,
    significance_threshold=0.001,)

Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: run_ancombc.R --inp_abundances_path /var/folders/hq/mt_9wjp90cx45qsgnk3d7qfc0000gq/T/tmpws79qhh8/input.biom.tsv --inp_metadata_path /var/folders/hq/mt_9wjp90cx45qsgnk3d7qfc0000gq/T/tmpws79qhh8/input.map.txt --md_column_types {"day-relative-to-fmt": "numeric", "PatientID": "categorical", "Timepoint": "numeric", "Consistency": "categorical", "Accession": "categorical", "BioProject": "categorical", "DayRelativeToNearestHCT": "numeric", "AccessionShotgun": "categorical", "patient-sample-counts": "numeric", "TimepointOfTransplant": "numeric", "HCTSource": "categorical", "Disease": "categorical", "wbcPatientId": "categorical", "autoFmtPatientId": "categorical", "nejmPatientId": "numeric", "autoFmtGroup": "categorical", "categorical-time-relative-to-hct"

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.0     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Registered S3 methods overwritten by 'treeio':
  method              from    
  MRCA.phylo          tidytree
  MRCA.treedata       tidytree
  Nnode.treedata      tidytree
  Ntip.treedata       tidytree
  ancestor.phylo      tidytree
  ancestor.treedata   tidytree
  child.phylo         tidytree
  child.treedata      tidytree
  full_join.phylo     tidytree
  full_join.treedata  tidytree
  groupClade.phylo    tidytree
  groupClade.treedata tidytree
  groupOTU.phylo      tidy

R version 4.2.2 (2022-10-31) 


'ancombc' is deprecated 
Use 'ancombc2' instead
`tax_level` is not speficified 
No agglomeration will be performed
Otherwise, please speficy `tax_level` by one of the following: 
Species


In [63]:
da_barplot_subject_viz

<visualization: Visualization uuid: a79d3df0-aecb-40af-b491-ec99e03f24b7>

In [70]:
sample_metadata1 = Metadata.load('2/sample-metadata1.tsv')
table=Artifact.load('2/table.qza')
gut_table=Artifact.load('2/gut-table.qza')
taxonomy=Artifact.load('2/taxonomy.qza')

In [66]:
ancombc_subject, = composition_actions.ancombc(
    table=gut_table,
    metadata=sample_metadata1,
    formula='subject',)

da_barplot_subject_viz, = composition_actions.da_barplot(
    data=ancombc_subject,
    significance_threshold=0.001,)

Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: run_ancombc.R --inp_abundances_path /var/folders/hq/mt_9wjp90cx45qsgnk3d7qfc0000gq/T/tmpmpmn58tv/input.biom.tsv --inp_metadata_path /var/folders/hq/mt_9wjp90cx45qsgnk3d7qfc0000gq/T/tmpmpmn58tv/input.map.txt --md_column_types {"barcode-sequence": "categorical", "body-site": "categorical", "year": "numeric", "month": "numeric", "day": "numeric", "subject": "categorical", "reported-antibiotic-usage": "categorical", "days-since-experiment-start": "numeric"} --formula subject --p_adj_method holm --prv_cut 0.1 --lib_cut 0 --reference_levels ['subject::subject-1'] --tol 1e-05 --max_iter 100 --conserve False --alpha 0.05 --output_loaf /var/folders/hq/mt_9wjp90cx45qsgnk3d7qfc0000gq/T/q2-DataLoafPackageDirFmt-5ulsm8l6



── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.0     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Registered S3 methods overwritten by 'treeio':
  method              from    
  MRCA.phylo          tidytree
  MRCA.treedata       tidytree
  Nnode.treedata      tidytree
  Ntip.treedata       tidytree
  ancestor.phylo      tidytree
  ancestor.treedata   tidytree
  child.phylo         tidytree
  child.treedata      tidytree
  full_join.phylo     tidytree
  full_join.treedata  tidytree
  groupClade.phylo    tidytree
  groupClade.treedata tidytree
  groupOTU.phylo      tidy

R version 4.2.2 (2022-10-31) 


'ancombc' is deprecated 
Use 'ancombc2' instead
`tax_level` is not speficified 
No agglomeration will be performed
Otherwise, please speficy `tax_level` by one of the following: 
Species


In [73]:
ancombc_subject.save('2/ancombc_subject.qza')
da_barplot_subject_viz.save('2/da_barplot_subject.qzv')

'2/da_barplot_subject.qzv'

In [67]:
da_barplot_subject_viz

<visualization: Visualization uuid: 9223bac3-6d24-4e67-8ee8-4c63c8bafc9a>

In [71]:
import qiime2.plugins.taxa.actions as taxa_actions
gut_table_l6, = taxa_actions.collapse(
    table=gut_table,
    taxonomy=taxonomy,
    level=6,)

l6_ancombc_subject, = composition_actions.ancombc(
    table=gut_table_l6,
    metadata=sample_metadata1,
    formula='subject',)

l6_da_barplot_subject_viz, = composition_actions.da_barplot(
    data=l6_ancombc_subject,
    significance_threshold=0.001,)

Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: run_ancombc.R --inp_abundances_path /var/folders/hq/mt_9wjp90cx45qsgnk3d7qfc0000gq/T/tmp9ba5jg0y/input.biom.tsv --inp_metadata_path /var/folders/hq/mt_9wjp90cx45qsgnk3d7qfc0000gq/T/tmp9ba5jg0y/input.map.txt --md_column_types {"barcode-sequence": "categorical", "body-site": "categorical", "year": "numeric", "month": "numeric", "day": "numeric", "subject": "categorical", "reported-antibiotic-usage": "categorical", "days-since-experiment-start": "numeric"} --formula subject --p_adj_method holm --prv_cut 0.1 --lib_cut 0 --reference_levels ['subject::subject-1'] --tol 1e-05 --max_iter 100 --conserve False --alpha 0.05 --output_loaf /var/folders/hq/mt_9wjp90cx45qsgnk3d7qfc0000gq/T/q2-DataLoafPackageDirFmt-tmhmkuaj



── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.0     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Registered S3 methods overwritten by 'treeio':
  method              from    
  MRCA.phylo          tidytree
  MRCA.treedata       tidytree
  Nnode.treedata      tidytree
  Ntip.treedata       tidytree
  ancestor.phylo      tidytree
  ancestor.treedata   tidytree
  child.phylo         tidytree
  child.treedata      tidytree
  full_join.phylo     tidytree
  full_join.treedata  tidytree
  groupClade.phylo    tidytree
  groupClade.treedata tidytree
  groupOTU.phylo      tidy

R version 4.2.2 (2022-10-31) 


'ancombc' is deprecated 
Use 'ancombc2' instead
`tax_level` is not speficified 
No agglomeration will be performed
Otherwise, please speficy `tax_level` by one of the following: 
Species


In [74]:
gut_table_l6.save('2/gut_table_l6.qza')
l6_ancombc_subject.save('2/l6_ancombc_subject.qza')
l6_da_barplot_subject_viz.save('2/l6_da_barplot_subject.qzv')

'l6_da_barplot_subject.qzv'

In [75]:
l6_da_barplot_subject_viz

<visualization: Visualization uuid: ff25209a-ea1b-4531-8e20-85a9f6da34e0>

In [ ]:
#Machine-learning classifiers

In [77]:
import qiime2.plugins.sample_classifier.actions as sample_classifier_actions

In [124]:
feature = Artifact.load('feature-table.qza')
metadata1 = Metadata.load('3/sample-metadata.tsv')

In [125]:
sample, = sample_classifier_actions.classify_samples(
    table=feature,
    metadata=metadata1.get_column('Disease_HCT'),
    random_state=666,
    n_jobs=4)

ValueError: You have chosen to predict a metadata column that contains one or more values that match only one sample. For proper stratification of data into training and test sets, each class (value) must contain at least two samples. This is a requirement for classification problems, but stratification can be disabled for regression by setting stratify=False. Alternatively, remove all samples that bear a unique class label for your chosen metadata column. Note that disabling stratification can negatively impact predictive accuracy for small data sets.